<h2 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px"> Tabular Playground Series - Aug 2021</h2> 



This is my first time participating in Tabular Playground series.I looked at the dataset and was compeltely lost as to where do I start , I had never worked with dataset with 100 columns. Second probelm since the features are anonymous , its very diffcult to make sense of data , relationship between features, deriving new features. Appreciate any kind of feedback.

<h2 style = "font-family:TimesNewRoman;color:black;font-weight:bold">Table of Contents</h2>

- [Introduction](#Introduction) 
- [Libraries](#Import-Libraries)
- [Read and Understand Data](#Read-and-Understand-Data)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis) 
- [Data Preparation](#Data-Preparation)
- [Model Building ](#Model-Building)


<h2 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">Introduction</h2> 


Kaggle competitions are incredibly fun and rewarding, but they can also be intimidating for people who are relatively new in their data science journey. In the past, Kaggle have launched many Playground competitions that are more approachable than Featured competition, and thus more beginner-friendly.

The goal of these competitions is to provide a fun, but less challenging, tabular dataset. These competitions will be great for people looking for something in between the Titanic Getting Started competition and a Featured competition.

The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with calculating the loss associated with a loan defaults. Although the features are anonymized, they have properties relating to real-world features.

**Task:Calculating the loss associated with a loan defaults.The features are anonymized, but they have properties relating to real-world features.**

**What is loan default?**

Default is a failure to repay a debt/loan on time. It can occur when a borrower fails to make timely payments on loans such as mortgage, bank loans, car leases, etc.

**Metric**

 Submissions are scored on the root mean squared error.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directo
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color:  #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">Import Libraries</h2> 

In [ ]:
#!pip install lightgbm
#!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler,QuantileTransformer,RobustScaler,PowerTransformer
from sklearn.model_selection import  RandomizedSearchCV
from sklearn import model_selection
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV,Lasso,LinearRegression
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgbm
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
    BaggingRegressor
)
from mlxtend.regressor import StackingCVRegressor
import warnings
warnings.filterwarnings('ignore') # To supress warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 90)
pd.set_option('display.max_colwidth',200)


<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color:  #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">Read and Understand Data</h2> 

In [ ]:
df_train=pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
df_test=pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")


In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print("#"*40,"\nTrain",)
print(f'There are {df_train.shape[0]} rows and {df_train.shape[1]} columns') # fstring 
#missing_df = pd.DataFrame({
#    "Missing": df_train.isnull().sum(),
#    "Missing %": round((df_train.isnull().sum()/ df_train.isna().count()*100), 2)
#})
#display(missing_df.sort_values(by='Missing', ascending=False))
print(f'There are {df_train.isnull().sum().mean()} missing values')

In [ ]:
print("#"*40,"\nTest",)
print(f'There are {df_test.shape[0]} rows and {df_test.shape[1]} columns') # fstring 
missing_df = pd.DataFrame({
    "Missing": df_test.isnull().sum(),
    "Missing %": round((df_test.isnull().sum()/ df_test.isna().count()*100), 2)
})
display(missing_df.sort_values(by='Missing', ascending=False))

print(f'There are {df_test.isnull().sum().mean()} missing values')

In [ ]:
#### Check the data types of the columns for the dataset.
df_train.info()

In [ ]:
#dropping id column
df_train.drop(['id'],inplace=True,axis=1)

In [ ]:
intfeatures=df_train.select_dtypes(include='int64')
intfeatures.columns

In [ ]:
floatfeatures=df_train.select_dtypes(include='float')
floatfeatures.columns

In [ ]:
floatfeatures.head(10)

In [ ]:
floatfeatures.tail(10)

In [ ]:
#### Check the data types of the columns for the dataset.
df_test.info()

<h4 style = "font-size:30px; font-family:Garamond ; font-weight : medium;  color :#852aa8   ; text-align: left; "><b>Observation </b></h4>

- Training set has 250K observations with 102 features .
- Testing set has 150K observations with 101 features
- `Loss` column is the target variable which is only available in the train dataset.
- There are no missing values in both sets
- In train dataset, 95 features are float64 and 7('id', 'f1', 'f16', 'f27', 'f55', 'f86', 'loss') are of  int64 type



<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color:  #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">Exploratory Data Analysis</h2> 

In [ ]:
# Colors to be used for plots
colors = ["#f72585","#b5179e","#7209b7","#560bad", "#3a0ca3", "#3f37c9", "#3f37c9",
          "#4361ee", "#4895ef", "#4895ef", "#4cc9f0"
          ]

In [ ]:
df_train.describe().T.style.bar(subset=['mean'], color='#f72585')\
                           .background_gradient(subset=['50%'], cmap='PiYG') # highlight median

In [ ]:
#checking f60 , it has very larges values .
df_train['f60']

In [ ]:
df = df_train.drop(["loss"],axis=1)
columns = df.columns.values
cols = 4
rows = len(columns) // cols + 1
fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(19,100), sharex=False) #subplot with all rows
plt.subplots_adjust(hspace = 0.4)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            axs[r,c].hist(df_train[columns[i]].values,
                                   color="#f72585",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset",bins=40)
            axs[r, c].set_title(columns[i], fontsize=17, pad=4)
            axs[r, c].tick_params(axis="y", labelsize=11)
            axs[r, c].tick_params(axis="x", labelsize=11)
           
        i+=1

plt.show();

Checking distribution of Target Varaible `Loss`

In [ ]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(df_train['loss'], color="purple");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="Loss")
ax.set(title="Loss distribution")
sns.despine(trim=True, left=True)
plt.show()

**Skewness**

It is the degree of distortion from the symmetrical bell curve or the normal distribution. It measures the lack of symmetry in data distribution.
It differentiates extreme values in one versus the other tail. A symmetrical distribution will have a skewness of 0.
 
**when is the skewness too much?**

The rule of thumb seems to be:
If the skewness is between -0.5 and 0.5, the data are fairly symmetrical.
If the skewness is between -1 and -0.5(negatively skewed) or between 0.5 and 1(positively skewed), the data are moderately skewed.
If the skewness is less than -1(negatively skewed) or greater than 1(positively skewed), the data are highly skewed.

**Kurtosis**
Kurtosis is all about the tails of the distribution — not the peakedness or flatness. It is used to describe the extreme values in one versus the other tail. It is actually the measure of outliers present in the distribution.Kurtosis > 3: Distribution is longer, tails are fatter. Peak is higher and sharper than Mesokurtic, which means that data are heavy-tailed or profusion of outliers.

In [ ]:
# Skew and kurt
print("Skewness: %f" % df_train['loss'].skew())
print("Kurtosis: %f" % df_train['loss'].kurt())

In [ ]:
# Find skewed numerical features
kurt_features = df_train[columns].kurt().sort_values(ascending=False)

high_kurt = kurt_features[kurt_features >3]
kurt_index = high_kurt.index

print("There are {} numerical features with kurtosis > 3 :".format(high_kurt.shape[0]))
skewness = pd.DataFrame({'Skew' :high_kurt})
kurt_features.head(10)

In [ ]:
# Find skewed numerical features
skew_features = df_train[columns].skew().sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
#checking intgers and some feature with bimodal  distrbution
plt.figure(figsize=(15,25))
columns=['f1','f3', 'f11','f12','f16','f18' ,'f27', 'f55', 'f86','f60','f76','f57','f84','f93','loss']
sns.pairplot(df_train[columns],corner=True)


In [ ]:
corr = df_train.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

plt.figure(figsize = (10, 10))
plt.title('Correlation matrix for Train data')
sns.heatmap(corr, mask = mask,  linewidths = .5,square=True,cbar_kws={"shrink": .60})
plt.show()

In [ ]:
#check correlation with target varaiable
loss_corr=corr['loss'].sort_values(ascending=False).head(10).to_frame()

cm = sns.light_palette("pink", as_cmap=True)

loss = loss_corr.style.background_gradient(cmap=cm)
loss

<h4 style = "font-size:30px; font-family:Garamond ; font-weight : medium;  color :#852aa8   ; text-align: left; "><b>Observation </b></h4>
- Most of the features are right skewed with outliers on higher end

- Target variable loss feature is rightskewed with outliers on higher end

- Some feautures are bimodal, trimodal.

- very few features have normal disturbution.
- few features have negative values
- f26, f86 can be converted to categorical features
- There is no significant correlation between features
- Instead of standardscaling which is not very robut at handling outliers we can trying using robust scaler / quantile transformer / power transform so data follows a normal distribution.





<h2 style = "font-size:35px; font-family:Garamond ; font-weight : normal; background-color:  #852aa8; color :#fed049   ; text-align: center; border-radius: 5px 5px; padding: 5px">Model Building</h2> 

In [ ]:
X = df_train.drop(['loss'], axis=1).values
y = df_train['loss'].values



**Here I am building different models using KFold and cross_val_score with pipelines and scaling with powertransform**

In [ ]:
# Setup cross validation folds
kf = KFold(n_splits=5, random_state=1, shuffle=True)

# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X_pass,y_pass):
    rmse = np.sqrt(-cross_val_score(model, X_pass, y_pass, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
   X, y, test_size=0.3, random_state=42
)

## Linear Regression,Lasso,ElasticNet

In [ ]:
# Initialize model using pipeline
pipe_lr = make_pipeline( PowerTransformer(), (LinearRegression()))

pipe_lasso = make_pipeline( PowerTransformer(), (Lasso(alpha=0.1)))

pipe_elastic = make_pipeline( PowerTransformer(), (ElasticNet(alpha=0.2,l1_ratio=0.6,random_state=1)))


In [ ]:

results = []  # Empty list to store all model's CV scores
names = []  # Empty list to store name of the models
scores={}
score = cv_rmse(pipe_lr,X_train,y_train)
results.append(score)
names.append("linear")
print("Linear: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['Linear'] = (score.mean(), score.std())
score = cv_rmse(pipe_lasso,X_train,y_train)
results.append(score)
names.append("lasso")
print("Lasso: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lasso'] = (score.mean(), score.std())
results.append(score)
names.append("elastic")
score = cv_rmse(pipe_elastic,X_train,y_train)
print("Elastic: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['Elastic'] = (score.mean(), score.std())


In [ ]:
# Plotting boxplots for CV scores of all models defined above
fig = plt.figure(figsize=(10, 7))

fig.suptitle("Algorithm Comparison")
ax = fig.add_subplot(111)

plt.boxplot(results)
ax.set_xticklabels(names)

plt.show()

In [ ]:
model_lr = pipe_lr.fit(X_train, y_train)
model_lr.score(X_valid, y_valid)
y_pred_lr = model_lr.predict(X_valid)
print('mean_absolute_error: ', mean_absolute_error(y_valid, y_pred_lr))
print('root_mean_squared_error: ', rmsle(y_valid, y_pred_lr))

In [ ]:
lasso_model = pipe_lasso.fit(X_train, y_train)
lasso_model.score(X_valid, y_valid)
y_pred_lasso = lasso_model.predict(X_valid)
print('mean_absolute_error: ', mean_absolute_error(y_valid, y_pred_lasso))
print('root_mean_squared_error: ', rmsle(y_valid, y_pred_lasso))

In [ ]:
elasticmod = pipe_elastic.fit(X_train, y_train)
elasticmod.score(X_valid,y_valid)

In [ ]:
y_pred_ela = elasticmod.predict(X_valid)
print('mean_absolute_error: ', mean_absolute_error(y_valid, y_pred_ela))
print('root_mean_squared_error: ', rmsle(y_valid, y_pred_ela))

In [ ]:
#fit on complete train data
final = pipe_lr.fit(X, y)

In [ ]:
finpreds = final.predict(df_test.drop('id', axis=1))

In [ ]:
df_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
df_sub['loss'] = finpreds
df_sub.to_csv('Linearsubmission.csv' , index = False)
df_sub.head(10)

**Just starting with simple model , and then will try feature selection ,converting few features in categorical, and some other models. If you like my Notebook please upvote,Thank you.**